"C:\Users\akhil\Downloads\best.pt"

In [3]:
import os
from roboflow import Roboflow

rf = Roboflow(api_key="bAfEOqSg064U08ToU7yY")
project = rf.workspace().project("weapon-detection-e6otc-ptny7")
dataset = project.version(1).download("yolov8")

DATASET_PATH = dataset.location
DATA_YAML = os.path.join(DATASET_PATH, "data.yaml")

print("Dataset downloaded to:", DATASET_PATH)


loading Roboflow workspace...
loading Roboflow project...
Dataset downloaded to: d:\AK1\AMRITA\SEM 8\OPEN LAB\vsc\weapon-detection-1


In [4]:
import yaml

with open(DATA_YAML, "r") as f:
    data = yaml.safe_load(f)

# Use training data as validation
data["val"] = data["train"]

with open(DATA_YAML, "w") as f:
    yaml.dump(data, f)

print("data.yaml fixed successfully")
print(data)

data.yaml fixed successfully
{'names': ['0', 'Knife', 'Person', 'Pistol', 'Sword', 'handgun', 'pistol', 'rifle'], 'nc': 8, 'roboflow': {'license': 'CC BY 4.0', 'project': 'weapon-detection-e6otc-ptny7', 'url': 'https://universe.roboflow.com/akhil-gr16i/weapon-detection-e6otc-ptny7/dataset/1', 'version': 1, 'workspace': 'akhil-gr16i'}, 'test': '../test/images', 'train': '../train/images', 'val': '../train/images'}


In [5]:
from ultralytics import YOLO
model = YOLO("C:/Users/akhil/Downloads/best.pt")

results = model("images.jpeg", conf=0.4)
results[0].show()


image 1/1 d:\AK1\AMRITA\SEM 8\OPEN LAB\vsc\images.jpeg: 640x576 1 Sword, 194.1ms
Speed: 4.4ms preprocess, 194.1ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 576)


THIS CODE IS FAR BETTER

In [6]:
import cv2

# Load trained model
print("Model classes:", model.names)

video_path = "1.mp4"  # or use 0 for webcam
cap = cv2.VideoCapture(video_path)

CONF_THRESH = 0.6       # strong confidence filtering
MIN_AREA = 3000        # removes pens, noise
TEMPORAL_FRAMES = 2    # stability

# Tracking
global_best_conf = 0.0
global_best_label = None
global_best_frame = None
frame_counts = {}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, conf=CONF_THRESH)
    annotated = results[0].plot()

    if results[0].boxes is not None:
        for box, cls_id, conf in zip(
            results[0].boxes.xyxy,
            results[0].boxes.cls,
            results[0].boxes.conf
        ):
            if conf < CONF_THRESH:
                continue

            x1, y1, x2, y2 = box
            area = (x2 - x1) * (y2 - y1)
            if area < MIN_AREA:
                continue

            label = model.names[int(cls_id)]

            # Temporal count
            frame_counts[label] = frame_counts.get(label, 0) + 1

            if frame_counts[label] >= TEMPORAL_FRAMES:
                if conf > global_best_conf:
                    global_best_conf = float(conf)
                    global_best_label = label
                    global_best_frame = frame.copy()

    cv2.imshow("Weapon Detection (Final)", annotated)
    if cv2.waitKey(25) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

# Final output
print("\n=== FINAL RESULT ===")
if global_best_frame is not None:
    print(f"Detected Weapon: {global_best_label}")
    print(f"Confidence: {global_best_conf:.2f}")

    cv2.imwrite(f"best_{global_best_label}.jpg", global_best_frame)
    cv2.imshow("Best Detection", global_best_frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("No reliable weapon detected.")


Model classes: {0: '0', 1: 'Knife', 2: 'Person', 3: 'Pistol', 4: 'Sword', 5: 'handgun', 6: 'pistol', 7: 'rifle'}

0: 640x384 (no detections), 169.8ms
Speed: 1.7ms preprocess, 169.8ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 173.6ms
Speed: 1.8ms preprocess, 173.6ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 136.6ms
Speed: 1.6ms preprocess, 136.6ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 130.3ms
Speed: 1.4ms preprocess, 130.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 126.8ms
Speed: 1.3ms preprocess, 126.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 121.3ms
Speed: 1.4ms preprocess, 121.3ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 121.4ms
Speed: 1.3ms preprocess, 121.4ms inferen

KeyboardInterrupt: 

Live video capturing

In [ ]:
# Load trained model
import cv2
print("Model classes:", model.names)

cap = cv2.VideoCapture(0)

CONF_THRESH = 0.6       # strong confidence filtering
MIN_AREA = 3000        # removes pens, noise
TEMPORAL_FRAMES = 2    # stability

frame_counts = {}
global_best_conf = 0.0
global_best_label = None
global_best_frame = None

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, conf=CONF_THRESH)
    annotated = results[0].plot()

    if results[0].boxes is not None:
        for box, cls_id, conf in zip(
            results[0].boxes.xyxy,
            results[0].boxes.cls,
            results[0].boxes.conf
        ):
            if conf < CONF_THRESH:
                continue

            x1, y1, x2, y2 = box
            area = (x2 - x1) * (y2 - y1)
            if area < MIN_AREA:
                continue

            label = model.names[int(cls_id)]

            frame_counts[label] = frame_counts.get(label, 0) + 1

            if frame_counts[label] >= TEMPORAL_FRAMES:
                if conf > global_best_conf:
                    global_best_conf = float(conf)
                    global_best_label = label
                    global_best_frame = frame.copy()

    cv2.putText(
        annotated,
        f"Best: {global_best_label} ({global_best_conf:.2f})"
        if global_best_label else "Scanning...",
        (20, 40),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 0, 255),
        2
    )

    cv2.imshow("Live Weapon Detection", annotated)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

# Save best frame
if global_best_frame is not None:
    cv2.imwrite(f"best_{global_best_label}_webcam.jpg", global_best_frame)
    print(f"Saved best frame: best_{global_best_label}_webcam.jpg")
else:
    print("No reliable weapon detected.")


NameError: name 'model' is not defined

this code gives alert and pop up message

In [ ]:
import cv2
import winsound
import threading
import time
import tkinter as tk
from tkinter import messagebox
from ultralytics import YOLO

# ================= CONFIG =================
DETECTION_CONF = 0.4      # allow detection
ALERT_CONF = 0.85         # strict alert threshold
ALARM_DURATION = 30       # seconds

# ================= LOAD MODEL =================
print("Model classes:", model.names)

# Weapon classes = everything EXCEPT person
WEAPON_CLASSES = {
    name.lower()
    for name in model.names.values()
    if name.lower() != "person"
}

cap = cv2.VideoCapture(0)

alert_active = False

# ================= ALERT FUNCTIONS =================
def popup_alert(label, conf):
    root = tk.Tk()
    root.withdraw()
    messagebox.showwarning(
        "🚨 WEAPON DETECTED",
        f"Weapon: {label.upper()}\nConfidence: {conf*100:.2f}%"
    )
    root.destroy()

def continuous_alarm():
    end_time = time.time() + ALARM_DURATION
    while time.time() < end_time:
        winsound.Beep(1200, 500)

# ================= MAIN LOOP =================
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 🔹 Detection is permissive
    results = model(frame, conf=DETECTION_CONF)
    annotated = results[0].plot()

    if results[0].boxes is not None:
        for cls_id, conf in zip(results[0].boxes.cls, results[0].boxes.conf):
            label = model.names[int(cls_id)].lower()

            # Ignore person
            if label not in WEAPON_CLASSES:
                continue

            # 🔴 Alert logic ONLY here
            if conf >= ALERT_CONF and not alert_active:
                alert_active = True
                print(f"🚨 ALERT: {label.upper()} ({conf:.2f})")

                threading.Thread(
                    target=popup_alert,
                    args=(label, conf),
                    daemon=True
                ).start()

                threading.Thread(
                    target=continuous_alarm,
                    daemon=True
                ).start()

    cv2.imshow("Live Weapon Detection", annotated)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


Model classes: {0: '0', 1: 'Knife', 2: 'Person', 3: 'Pistol', 4: 'Sword', 5: 'handgun', 6: 'pistol', 7: 'rifle'}

0: 480x640 (no detections), 166.7ms
Speed: 1.9ms preprocess, 166.7ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 151.9ms
Speed: 2.3ms preprocess, 151.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 145.3ms
Speed: 2.0ms preprocess, 145.3ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 152.7ms
Speed: 1.6ms preprocess, 152.7ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 142.5ms
Speed: 1.7ms preprocess, 142.5ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 140.6ms
Speed: 1.9ms preprocess, 140.6ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 148.8ms
Speed: 1.4ms preprocess, 148.8ms inferen

KeyboardInterrupt: 

: 